# Importation et inspection des données avis_restaurants.csv

In [2]:
from db.models import get_user_and_review_from_restaurant_id 
from db.models import get_all_restaurants

# Connexion à la base de données
engine = create_engine('sqlite:///restaurant_reviews.db')
Session = sessionmaker(bind=engine)
session = Session()

restaurants = get_all_restaurants(session)

for restaurant in restaurants:
    avis_restaurants = get_user_and_review_from_restaurant_id(session, restaurant.id_restaurant)

ModuleNotFoundError: No module named 'db'

In [ ]:
print(avis_restaurants.info())

In [69]:
import pandas as pd
# Charger les données
#avis_restaurants = pd.read_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/avis_restaurants.csv', sep=';') 

# Aperçu des données
#print(avis_restaurants.head())

               user      user_profile       date_review  \
0            Fred S      fredsX5996LQ  15 décembre 2024   
1  Guide38409630404  Guide38409630404  27 novembre 2024   
2         Chartin M          chartinm  20 novembre 2024   
3      Axelfoley009      Axelfoley009  18 novembre 2024   
4          Karine P     karinepU103AS  13 novembre 2024   

                                  title  rating type_visit  num_contributions  \
0                 Très belle expérience     5.0     family                  8   
1  Déjeuner au Frazarin, que du bonheur     5.0    friends                  1   
2         La perfection tout simplement     5.0    couples                  4   
3              Plat frais et de qualité     5.0    couples                 38   
4                             très bien     5.0   business                  1   

                                              review  \
0  Ambiance soignée, patrons amicaux et donnant t...   
1  Vous voulez bien manger, alors allez dans c

In [70]:
print(avis_restaurants.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4125 entries, 0 to 4124
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               4125 non-null   object 
 1   user_profile       4125 non-null   object 
 2   date_review        4125 non-null   object 
 3   title              4125 non-null   object 
 4   rating             4125 non-null   float64
 5   type_visit         4125 non-null   object 
 6   num_contributions  4125 non-null   int64  
 7   review             4125 non-null   object 
 8   restaurant         4125 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 290.2+ KB
None


# Nettoyage des avis

Inclure des étapes pour retirer les caractères inutiles, convertir en minuscules, supprimer les stopwords, etc.

In [71]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [72]:
def nettoyer_avis(avis):
    # Retirer les caractères spéciaux et convertir en minuscules
    avis = avis.lower()
    avis = re.sub(r'[^\w\s]', '', avis)
    
    # retirer les chiffres
    avis = re.sub(r'[0-9]', '', avis)
    
    # Tokenisation
    tokens = word_tokenize(avis)
    
    # lematisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Retirer les stopwords
    tokens = [word for word in tokens if word not in stopwords.words('french')]
    
    # retirer les termes de moins de 3 caractères
    tokens = [word for word in tokens if len(word) > 2]
    
    return ' '.join(tokens)

In [73]:
# Appliquer le nettoyage à la colonne 'review'
avis_restaurants['review_cleaned'] = avis_restaurants['review'].apply(nettoyer_avis)
print(avis_restaurants[['review', 'review_cleaned']].head())

                                              review  \
0  Ambiance soignée, patrons amicaux et donnant t...   
1  Vous voulez bien manger, alors allez dans ce r...   
2  Tout simplement parfait de l’entrée au dessert...   
3  Belle soirée passée au Frazarin. Certainement ...   
4  Très bon restaurant ! j'ai adoré le service et...   

                                      review_cleaned  
0  ambiance soignée patron amicaux donnant toutes...  
1  voulez bien manger alors allez restaurant tout...  
2  tout simplement parfait lentrée dessert passan...  
3  belle soirée passée frazarin certainement enco...  
4  très bon restaurant jai adoré service loffre r...  


# Création de représentations textuelles

Transforme les avis en vecteurs numériques pour les modèles.

## Bag of Words

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorisation BoW
vectorizer = CountVectorizer(max_features=500)  # Limiter à 500 mots les plus fréquents
X_bow = vectorizer.fit_transform(avis_restaurants['review_cleaned'])
print(X_bow.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## TF-IDF

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = tfidf_vectorizer.fit_transform(avis_restaurants['review_cleaned'])
print(X_tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Préparation pour l'analyse temporelle

In [76]:
avis_restaurants['date_review']

0       15 décembre 2024
1       27 novembre 2024
2       20 novembre 2024
3       18 novembre 2024
4       13 novembre 2024
              ...       
4120     4 novembre 2016
4121     25 octobre 2016
4122     16 octobre 2016
4123     13 octobre 2016
4124     10 octobre 2016
Name: date_review, Length: 4125, dtype: object

In [77]:
avis_restaurants['date_review'] = avis_restaurants['date_review'].str.strip()
avis_restaurants['date_review'] = avis_restaurants['date_review'].str.replace(r'\s+', ' ', regex=True)

In [78]:
print(avis_restaurants['date_review'].head(10))


0     15 décembre 2024
1     27 novembre 2024
2     20 novembre 2024
3     18 novembre 2024
4     13 novembre 2024
5     13 novembre 2024
6      22 octobre 2024
7      17 octobre 2024
8      11 octobre 2024
9    26 septembre 2024
Name: date_review, dtype: object


In [79]:
mois_fr_en = {
    'janvier': 'January', 'février': 'February', 'mars': 'March', 'avril': 'April',
    'mai': 'May', 'juin': 'June', 'juillet': 'July', 'août': 'August',
    'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'décembre': 'December'
}

for fr, en in mois_fr_en.items():
    avis_restaurants['date_review'] = avis_restaurants['date_review'].str.replace(fr, en, regex=False)


In [80]:
avis_restaurants['date_review']

0       15 December 2024
1       27 November 2024
2       20 November 2024
3       18 November 2024
4       13 November 2024
              ...       
4120     4 November 2016
4121     25 October 2016
4122     16 October 2016
4123     13 October 2016
4124     10 October 2016
Name: date_review, Length: 4125, dtype: object

In [81]:
avis_restaurants['date_review'] = pd.to_datetime(
    avis_restaurants['date_review'], format='%d %B %Y', errors='coerce'
)


In [82]:
print(avis_restaurants['date_review'].isna().sum())


0


In [83]:
problemes = avis_restaurants[avis_restaurants['date_review'].isna()]
print(problemes['date_review'])


Series([], Name: date_review, dtype: datetime64[ns])


In [84]:
avis_restaurants['date_review']

0      2024-12-15
1      2024-11-27
2      2024-11-20
3      2024-11-18
4      2024-11-13
          ...    
4120   2016-11-04
4121   2016-10-25
4122   2016-10-16
4123   2016-10-13
4124   2016-10-10
Name: date_review, Length: 4125, dtype: datetime64[ns]

In [85]:
# Extraire l'année et le mois pour une analyse temporelle
avis_restaurants['year'] = avis_restaurants['date_review'].dt.year
avis_restaurants['month'] = avis_restaurants['date_review'].dt.month
print(avis_restaurants[['date_review', 'year', 'month']].head())

  date_review  year  month
0  2024-12-15  2024     12
1  2024-11-27  2024     11
2  2024-11-20  2024     11
3  2024-11-18  2024     11
4  2024-11-13  2024     11


# Sauvegarder le dataset nettoyé

In [86]:
avis_restaurants.to_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/avis_restaurants_cleaned.csv', index=False)

In [87]:
avis_restaurants_cleaned = pd.read_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/avis_restaurants_cleaned.csv') 

In [88]:
avis_restaurants_cleaned

,user,user_profile,date_review,title,rating,type_visit,num_contributions,review,restaurant,review_cleaned,year,month
0,Fred S,fredsX5996LQ,2024-12-15,Très belle expérience,5.0,family,8,"Ambiance soignée, patrons amicaux et donnant t...",Frazarin Bistrot Franco Italien,ambiance soignée patron amicaux donnant toutes...,2024,12
1,Guide38409630404,Guide38409630404,2024-11-27,"Déjeuner au Frazarin, que du bonheur",5.0,friends,1,"Vous voulez bien manger, alors allez dans ce r...",Frazarin Bistrot Franco Italien,voulez bien manger alors allez restaurant tout...,2024,11
2,Chartin M,chartinm,2024-11-20,La perfection tout simplement,5.0,couples,4,Tout simplement parfait de l’entrée au dessert...,Frazarin Bistrot Franco Italien,tout simplement parfait lentrée dessert passan...,2024,11
3,Axelfoley009,Axelfoley009,2024-11-18,Plat frais et de qualité,5.0,couples,38,Belle soirée passée au Frazarin. Certainement ...,Frazarin Bistrot Franco Italien,belle soirée passée frazarin certainement enco...,2024,11
4,Karine P,karinepU103AS,2024-11-13,très bien,5.0,business,1,Très bon restaurant ! j'ai adoré le service et...,Frazarin Bistrot Franco Italien,très bon restaurant jai adoré service loffre r...,2024,11
...,...,...,...,...,...,...,...,...,...,...,...,...
4120,Muriel P,752murielp,2016-11-04,Des troyens visitent Lyon,4.0,family,6,Bon petit resto. Service efficace et agréable....,Le Vieux Lyon,bon petit resto service efficace agréable cuis...,2016,11
4121,Natacha B,natachab927,2016-10-25,Génial!,5.0,family,3,Un super service! Avec de très bons plats😊 rec...,Le Vieux Lyon,super service très bons plat recommandé sans h...,2016,10
4122,Loïc D,lo_cd778,2016-10-16,tres bien,4.0,couples,35,"Acceuil sympa , notre serveur dans le jus mais...",Le Vieux Lyon,acceuil sympa serveur jus très efficace cétait...,2016,10
4123,357518,357518,2016-10-13,"Touristes, mais aussi habitués !",4.0,none,30,"De loin, le meilleur petit restau de la rue......",Le Vieux Lyon,loin meilleur petit restau rue cuisine familia...,2016,10


# Importation et inspection des données La_Mere_Brazier.csv

In [89]:
mere_brazier = pd.read_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/La_Mere_Brazier.csv', sep=';')

mere_brazier

,user,user_profile,date_review,title,rating,type_visit,num_contributions,review
0,Serpollet,Serpollet,2 décembre 2024,Grande tradition lyonnaise,5.0,couples,152,Nous avons passé un excellent moment dans ce r...
1,tif s,925tifs,18 novembre 2024,Bon resto,5.0,family,17,Excellent restaurant \nSuper qualité prix \nNo...
2,ninoacmo,ninoacmo,15 novembre 2024,"Merveilleux repas, service parfait",5.0,couples,30,Notre fils nous a offert un menu Eugénie Brazi...
3,Djill38,Djill38,21 octobre 2024,Une explosion de saveurs...,5.0,couples,125,De la prise en charge dès la porte poussée et ...
4,LILI,Odyssey17905604830,24 août 2024,ADRESSE À FAIRE UNE FOIS,3.0,couples,237,La décoration est resté (beaucoup trop) figé d...
...,...,...,...,...,...,...,...,...
910,janicejmf,janicejmf,6 décembre 2012,Expérience merveilleuse de la salle à manger,5.0,friends,13,Quatre d'entre nous avons voyagé par TGV Agde ...
911,theorchidlady,theorchidlady,18 octobre 2012,Deux excellent mais cher restaurant Star,5.0,family,112,Mon mari et moi avons visité ce restaurant deu...
912,Trav3lgur,Trav3lgur,2 septembre 2012,Si je pouvais donner 6 étoiles !! Pour ne pas ...,5.0,No information,178,"J'ai déjeuné ici comme un voyageur en solo, c'..."
913,daretobe99999,daretobe99999,26 juillet 2012,J'ai été impressionné ...,5.0,business,12,J'ai été impressionné par la belle présentatio...


In [90]:
print(mere_brazier.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               915 non-null    object 
 1   user_profile       915 non-null    object 
 2   date_review        915 non-null    object 
 3   title              915 non-null    object 
 4   rating             915 non-null    float64
 5   type_visit         915 non-null    object 
 6   num_contributions  915 non-null    int64  
 7   review             915 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 57.3+ KB
None


# Nettoyage des avis

In [91]:
# Appliquer le nettoyage à la colonne 'review'
mere_brazier['review_cleaned'] = mere_brazier['review'].apply(nettoyer_avis)
print(mere_brazier[['review', 'review_cleaned']].head())

                                              review  \
0  Nous avons passé un excellent moment dans ce r...   
1  Excellent restaurant \nSuper qualité prix \nNo...   
2  Notre fils nous a offert un menu Eugénie Brazi...   
3  De la prise en charge dès la porte poussée et ...   
4  La décoration est resté (beaucoup trop) figé d...   

                                      review_cleaned  
0  avon passé excellent moment restaurant réputat...  
1  excellent restaurant super qualité prix nourri...  
2  fils offert menu eugénie brazier deux arrivée ...  
3  prise charge dès porte poussée jusquà sortie c...  
4  décoration resté beaucoup trop figé temp pourt...  


# Préparation des représentations textuelles

## Bag of Words

In [92]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorisation BoW
vectorizer = CountVectorizer(max_features=500)
X_bow = vectorizer.fit_transform(mere_brazier['review_cleaned'])
print(X_bow.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


## TF-IDF

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = tfidf_vectorizer.fit_transform(mere_brazier['review_cleaned'])
print(X_tfidf.toarray())


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.21072479 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.06917037]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.14169917 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# Préparation pour l'analyse temporelle

In [94]:
mere_brazier['date_review']

0       2 décembre 2024
1      18 novembre 2024
2      15 novembre 2024
3       21 octobre 2024
4          24 août 2024
             ...       
910     6 décembre 2012
911     18 octobre 2012
912    2 septembre 2012
913     26 juillet 2012
914     11 juillet 2012
Name: date_review, Length: 915, dtype: object

In [95]:
mere_brazier['date_review'] = mere_brazier['date_review'].str.strip()
mere_brazier['date_review'] = mere_brazier['date_review'].str.replace(r'\s+', ' ', regex=True)

In [96]:
print(mere_brazier['date_review'].head(10))

0     2 décembre 2024
1    18 novembre 2024
2    15 novembre 2024
3     21 octobre 2024
4        24 août 2024
5        26 juin 2024
6        10 juin 2024
7          8 mai 2024
8       29 avril 2024
9        17 mars 2024
Name: date_review, dtype: object


In [97]:
for fr, en in mois_fr_en.items():
    mere_brazier['date_review'] = mere_brazier['date_review'].str.replace(fr, en, regex=False)


In [98]:
mere_brazier['date_review']

0       2 December 2024
1      18 November 2024
2      15 November 2024
3       21 October 2024
4        24 August 2024
             ...       
910     6 December 2012
911     18 October 2012
912    2 September 2012
913        26 July 2012
914        11 July 2012
Name: date_review, Length: 915, dtype: object

In [99]:
mere_brazier['date_review'] = pd.to_datetime(
    mere_brazier['date_review'], format='%d %B %Y', errors='coerce'
)

In [100]:
print(mere_brazier['date_review'].isna().sum())

0


In [101]:
problemes = mere_brazier[mere_brazier['date_review'].isna()]
print(problemes['date_review'])


Series([], Name: date_review, dtype: datetime64[ns])


In [102]:
mere_brazier['date_review']

0     2024-12-02
1     2024-11-18
2     2024-11-15
3     2024-10-21
4     2024-08-24
         ...    
910   2012-12-06
911   2012-10-18
912   2012-09-02
913   2012-07-26
914   2012-07-11
Name: date_review, Length: 915, dtype: datetime64[ns]

In [103]:
# Extraire l'année et le mois pour une analyse temporelle
mere_brazier['year'] = mere_brazier['date_review'].dt.year
mere_brazier['month'] = mere_brazier['date_review'].dt.month
print(mere_brazier[['date_review', 'year', 'month']].head(10))

  date_review  year  month
0  2024-12-02  2024     12
1  2024-11-18  2024     11
2  2024-11-15  2024     11
3  2024-10-21  2024     10
4  2024-08-24  2024      8
5  2024-06-26  2024      6
6  2024-06-10  2024      6
7  2024-05-08  2024      5
8  2024-04-29  2024      4
9  2024-03-17  2024      3


# Sauvegarde des données nettoyées

In [104]:
mere_brazier.to_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/La_Mere_Brazier_cleaned.csv', index=False)

In [105]:
mere_brazier_cleaned = pd.read_csv('D:/M2 SISE/Text Mining ou NLP/Projet/nlp_text_mining/Data/La_Mere_Brazier_cleaned.csv')

In [106]:
mere_brazier_cleaned

,user,user_profile,date_review,title,rating,type_visit,num_contributions,review,review_cleaned,year,month
0,Serpollet,Serpollet,2024-12-02,Grande tradition lyonnaise,5.0,couples,152,Nous avons passé un excellent moment dans ce r...,avon passé excellent moment restaurant réputat...,2024,12
1,tif s,925tifs,2024-11-18,Bon resto,5.0,family,17,Excellent restaurant \nSuper qualité prix \nNo...,excellent restaurant super qualité prix nourri...,2024,11
2,ninoacmo,ninoacmo,2024-11-15,"Merveilleux repas, service parfait",5.0,couples,30,Notre fils nous a offert un menu Eugénie Brazi...,fils offert menu eugénie brazier deux arrivée ...,2024,11
3,Djill38,Djill38,2024-10-21,Une explosion de saveurs...,5.0,couples,125,De la prise en charge dès la porte poussée et ...,prise charge dès porte poussée jusquà sortie c...,2024,10
4,LILI,Odyssey17905604830,2024-08-24,ADRESSE À FAIRE UNE FOIS,3.0,couples,237,La décoration est resté (beaucoup trop) figé d...,décoration resté beaucoup trop figé temp pourt...,2024,8
...,...,...,...,...,...,...,...,...,...,...,...
910,janicejmf,janicejmf,2012-12-06,Expérience merveilleuse de la salle à manger,5.0,friends,13,Quatre d'entre nous avons voyagé par TGV Agde ...,quatre dentre avon voyagé tgv agde lyon avon d...,2012,12
911,theorchidlady,theorchidlady,2012-10-18,Deux excellent mais cher restaurant Star,5.0,family,112,Mon mari et moi avons visité ce restaurant deu...,mari avon visité restaurant deux étoiles miche...,2012,10
912,Trav3lgur,Trav3lgur,2012-09-02,Si je pouvais donner 6 étoiles !! Pour ne pas ...,5.0,No information,178,"J'ai déjeuné ici comme un voyageur en solo, c'...",jai déjeuné ici comme voyageur solo cétait lun...,2012,9
913,daretobe99999,daretobe99999,2012-07-26,J'ai été impressionné ...,5.0,business,12,J'ai été impressionné par la belle présentatio...,jai impressionné belle présentation nourriture...,2012,7
